### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
print(groq_api_key)

from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

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


In [2]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [3]:
chain=prompt|model
chain.invoke({"messages":[HumanMessage(content="Hi My name is ram")]})

AIMessage(content="Hello Ram! 👋  It's nice to meet you.\n\nWhat can I help you with today?  I'm ready for any questions you have! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 31, 'total_tokens': 70, 'completion_time': 0.070909091, 'prompt_time': 0.002393405, 'queue_time': 0.296355074, 'total_time': 0.073302496}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-87adfd6a-8212-421d-8a1d-bff2d966e270-0', usage_metadata={'input_tokens': 31, 'output_tokens': 39, 'total_tokens': 70})

In [4]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is ram")],
    config=config
)

response

AIMessage(content="Hi Ram! It's nice to meet you.\n\nWhat can I do for you today? I'm ready to answer your questions to the best of my ability.  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 31, 'total_tokens': 72, 'completion_time': 0.074545455, 'prompt_time': 0.002368075, 'queue_time': 0.018608344, 'total_time': 0.07691353}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8da99531-f550-4d56-9002-16654421b50f-0', usage_metadata={'input_tokens': 31, 'output_tokens': 41, 'total_tokens': 72})

In [5]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ram.  😊  \n\nHow can I help you further?\n'

In [6]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [7]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is ram")],"language":"Hindi"})
response.content

'नमस्ते राम! \n\nआप मुझसे कुछ पूछना चाहते हैं? मैं आपकी मदद करने के लिए उत्सुक हूँ। 😊  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.


In [8]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [10]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am bharat")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते भारत! \n\nआप कैसे हैं? क्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

In [11]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम भारत है। \n'